In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Indian Stock Market Correlation Study\n",
    "## Market Research System v1.0 - Exploratory Data Analysis\n",
    "\n",
    "**Created:** January 2022  \n",
    "**Last Updated:** December 2022  \n",
    "**Focus:** Indian Stock Market (NSE/BSE)  \n",
    "**Objective:** Analyze correlations between major Indian stocks, sectors, and economic indicators\n",
    "\n",
    "---"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Setup and Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages in Google Colab\n",
    "!pip install yfinance pandas numpy matplotlib seaborn plotly scipy\n",
    "!pip install nsepy  # For Indian stock data\n",
    "\n",
    "# Import libraries\n",
    "import yfinance as yf\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.graph_objects as go\n",
    "import plotly.express as px\n",
    "from plotly.subplots import make_subplots\n",
    "from scipy import stats\n",
    "from datetime import datetime, timedelta\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set display options\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.max_rows', 100)\n",
    "\n",
    "# Plotting style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"✅ All dependencies loaded successfully!\")\n",
    "print(f\"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🏢 Indian Stock Universe Definition (2022 Focus)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define Indian stock universe (NSE symbols with .NS suffix for Yahoo Finance)\n",
    "NIFTY_50_STOCKS = {\n",
    "    # Technology Sector\n",
    "    'TCS.NS': 'Tata Consultancy Services',\n",
    "    'INFY.NS': 'Infosys',\n",
    "    'HCLTECH.NS': 'HCL Technologies',\n",
    "    'WIPRO.NS': 'Wipro',\n",
    "    'TECHM.NS': 'Tech Mahindra',\n",
    "    'LTI.NS': 'L&T Infotech',\n",
    "    \n",
    "    # Banking & Financial Services\n",
    "    'HDFCBANK.NS': 'HDFC Bank',\n",
    "    'ICICIBANK.NS': 'ICICI Bank',\n",
    "    'SBIN.NS': 'State Bank of India',\n",
    "    'AXISBANK.NS': 'Axis Bank',\n",
    "    'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
    "    'INDUSINDBK.NS': 'IndusInd Bank',\n",
    "    'BAJFINANCE.NS': 'Bajaj Finance',\n",
    "    'HDFCLIFE.NS': 'HDFC Life Insurance',\n",
    "    'SBILIFE.NS': 'SBI Life Insurance',\n",
    "    \n",
    "    # Energy & Oil\n",
    "    'RELIANCE.NS': 'Reliance Industries',\n",
    "    'ONGC.NS': 'Oil & Natural Gas Corp',\n",
    "    'BPCL.NS': 'Bharat Petroleum',\n",
    "    'IOC.NS': 'Indian Oil Corporation',\n",
    "    'GAIL.NS': 'GAIL India',\n",
    "    \n",
    "    # Automobiles\n",
    "    'MARUTI.NS': 'Maruti Suzuki',\n",
    "    'M&M.NS': 'Mahindra & Mahindra',\n",
    "    'TATAMOTORS.NS': 'Tata Motors',\n",
    "    'BAJAJ-AUTO.NS': 'Bajaj Auto',\n",
    "    'HEROMOTOCO.NS': 'Hero MotoCorp',\n",
    "    \n",
    "    # FMCG\n",
    "    'HINDUNILVR.NS': 'Hindustan Unilever',\n",
    "    'ITC.NS': 'ITC Limited',\n",
    "    'NESTLEIND.NS': 'Nestle India',\n",
    "    'BRITANNIA.NS': 'Britannia Industries',\n",
    "    'DABUR.NS': 'Dabur India',\n",
    "    \n",
    "    # Pharmaceuticals\n",
    "    'SUNPHARMA.NS': 'Sun Pharmaceutical',\n",
    "    'DRREDDY.NS': 'Dr. Reddys Laboratories',\n",
    "    'CIPLA.NS': 'Cipla',\n",
    "    'DIVISLAB.NS': 'Divis Laboratories',\n",
    "    'BIOCON.NS': 'Biocon',\n",
    "    \n",
    "    # Metals & Mining\n",
    "    'TATASTEEL.NS': 'Tata Steel',\n",
    "    'HINDALCO.NS': 'Hindalco Industries',\n",
    "    'JSW.NS': 'JSW Steel',\n",
    "    'COALINDIA.NS': 'Coal India',\n",
    "    'VEDL.NS': 'Vedanta Limited',\n",
    "    \n",
    "    # Infrastructure & Construction\n",
    "    'LT.NS': 'Larsen & Toubro',\n",
    "    'ULTRACEMCO.NS': 'UltraTech Cement',\n",
    "    'GRASIM.NS': 'Grasim Industries',\n",
    "    'ADANIPORTS.NS': 'Adani Ports',\n",
    "    \n",
    "    # Telecom\n",
    "    'BHARTIARTL.NS': 'Bharti Airtel',\n",
    "    'JIO.NS': 'Reliance Jio' # Note: May not be available in 2022\n",
    "}\n",
    "\n",
    "# Market Indices\n",
    "INDIAN_INDICES = {\n",
    "    '^NSEI': 'NIFTY 50',\n",
    "    '^BSESN': 'BSE SENSEX',\n",
    "    '^NSEBANK': 'NIFTY Bank',\n",
    "    '^NSEIT': 'NIFTY IT',\n",
    "    'NIFTYFMCG.NS': 'NIFTY FMCG',\n",
    "    'CNXAUTO.NS': 'NIFTY Auto'\n",
    "}\n",
    "\n",
    "# Sector Classification\n",
    "SECTORS = {\n",
    "    'Technology': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS', 'TECHM.NS', 'LTI.NS'],\n",
    "    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS', 'AXISBANK.NS', 'KOTAKBANK.NS', 'INDUSINDBK.NS'],\n",
    "    'Energy': ['RELIANCE.NS', 'ONGC.NS', 'BPCL.NS', 'IOC.NS', 'GAIL.NS'],\n",
    "    'Automobiles': ['MARUTI.NS', 'M&M.NS', 'TATAMOTORS.NS', 'BAJAJ-AUTO.NS', 'HEROMOTOCO.NS'],\n",
    "    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'NESTLEIND.NS', 'BRITANNIA.NS', 'DABUR.NS'],\n",
    "    'Pharmaceuticals': ['SUNPHARMA.NS', 'DRREDDY.NS', 'CIPLA.NS', 'DIVISLAB.NS', 'BIOCON.NS'],\n",
    "    'Metals': ['TATASTEEL.NS', 'HINDALCO.NS', 'JSW.NS', 'COALINDIA.NS', 'VEDL.NS'],\n",
    "    'Infrastructure': ['LT.NS', 'ULTRACEMCO.NS', 'GRASIM.NS', 'ADANIPORTS.NS']\n",
    "}\n",
    "\n",
    "print(f\"📈 Total stocks to analyze: {len(NIFTY_50_STOCKS)}\")\n",
    "print(f\"🏭 Sectors defined: {len(SECTORS)}\")\n",
    "print(f\"📊 Indices to track: {len(INDIAN_INDICES)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📅 Data Collection Period Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define analysis period (2022 focus)\n",
    "START_DATE = '2020-01-01'  # Historical context\n",
    "END_DATE = '2022-12-31'    # 2022 end\n",
    "\n",
    "# For correlation analysis - different time windows\n",
    "CORRELATION_PERIODS = {\n",
    "    'Full Period': (START_DATE, END_DATE),\n",
    "    '2022 Only': ('2022-01-01', '2022-12-31'),\n",
    "    'COVID Recovery': ('2021-01-01', '2022-12-31'),\n",
    "    'Last 6 Months': ('2022-07-01', '2022-12-31'),\n",
    "    'Last 3 Months': ('2022-10-01', '2022-12-31')\n",
    "}\n",
    "\n",
    "print(f\"📊 Analysis Period: {START_DATE} to {END_DATE}\")\n",
    "print(f\"🔍 Correlation windows: {list(CORRELATION_PERIODS.keys())}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💾 Data Fetching Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def fetch_stock_data(symbols, start_date, end_date, column='Adj Close'):\n",
    "    \"\"\"\n",
    "    Fetch stock data for multiple symbols from Yahoo Finance\n",
    "    \"\"\"\n",
    "    data = pd.DataFrame()\n",
    "    failed_symbols = []\n",
    "    \n",
    "    print(f\"📥 Fetching data for {len(symbols)} symbols...\")\n",
    "    \n",
    "    for i, symbol in enumerate(symbols):\n",
    "        try:\n",
    "            stock = yf.Ticker(symbol)\n",
    "            hist = stock.history(start=start_date, end=end_date)\n",
    "            \n",
    "            if not hist.empty and len(hist) > 50:  # Minimum data points\n",
    "                data[symbol] = hist[column]\n",
    "                print(f\"✅ {symbol}: {len(hist)} data points\")\n",
    "            else:\n",
    "                failed_symbols.append(symbol)\n",
    "                print(f\"❌ {symbol}: Insufficient data\")\n",
    "                \n",
    "        except Exception as e:\n",
    "            failed_symbols.append(symbol)\n",
    "            print(f\"❌ {symbol}: Error - {str(e)}\")\n",
    "    \n",
    "    print(f\"\\n📊 Successfully fetched: {len(data.columns)} stocks\")\n",
    "    print(f\"❌ Failed: {len(failed_symbols)} stocks\")\n",
    "    \n",
    "    if failed_symbols:\n",
    "        print(f\"Failed symbols: {failed_symbols}\")\n",
    "    \n",
    "    return data.dropna()\n",
    "\n",
    "def calculate_returns(price_data, method='daily'):\n",
    "    \"\"\"\n",
    "    Calculate returns from price data\n",
    "    \"\"\"\n",
    "    if method == 'daily':\n",
    "        returns = price_data.pct_change().dropna()\n",
    "    elif method == 'weekly':\n",
    "        weekly_prices = price_data.resample('W').last()\n",
    "        returns = weekly_prices.pct_change().dropna()\n",
    "    elif method == 'monthly':\n",
    "        monthly_prices = price_data.resample('M').last()\n",
    "        returns = monthly_prices.pct_change().dropna()\n",
    "    else:\n",
    "        raise ValueError(\"Method must be 'daily', 'weekly', or 'monthly'\")\n",
    "    \n",
    "    return returns\n",
    "\n",
    "def clean_symbol_names(data, symbol_mapping):\n",
    "    \"\"\"\n",
    "    Clean symbol names for better visualization\n",
    "    \"\"\"\n",
    "    clean_data = data.copy()\n",
    "    \n",
    "    # Rename columns to company names\n",
    "    rename_dict = {}\n",
    "    for symbol in clean_data.columns:\n",
    "        if symbol in symbol_mapping:\n",
    "            company_name = symbol_mapping[symbol]\n",
    "            # Shorten long names\n",
    "            if len(company_name) > 15:\n",
    "                company_name = company_name.split()[0]  # Take first word\n",
    "            rename_dict[symbol] = company_name\n",
    "        else:\n",
    "            rename_dict[symbol] = symbol.replace('.NS', '')\n",
    "    \n",
    "    clean_data.rename(columns=rename_dict, inplace=True)\n",
    "    return clean_data\n",
    "\n",
    "print(\"🔧 Data fetching functions defined successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Main Data Collection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fetch stock price data\n",
    "print(\"🚀 Starting data collection...\\n\")\n",
    "\n",
    "# Get all stock symbols\n",
    "all_symbols = list(NIFTY_50_STOCKS.keys())\n",
    "\n",
    "# Remove potentially unavailable symbols for 2022\n",
    "available_symbols = [s for s in all_symbols if s != 'JIOL.NS']  # Jio might not be listed in 2022\n",
    "\n",
    "# Fetch price data\n",
    "price_data = fetch_stock_data(available_symbols, START_DATE, END_DATE)\n",
    "\n",
    "print(f\"\\n📈 Price data shape: {price_data.shape}\")\n",
    "print(f\"📅 Date range: {price_data.index[0].date()} to {price_data.index[-1].date()}\")\n",
    "\n",
    "# Calculate daily returns\n",
    "daily_returns = calculate_returns(price_data, 'daily')\n",
    "print(f\"📊 Daily returns shape: {daily_returns.shape}\")\n",
    "\n",
    "# Calculate weekly and monthly returns for different correlation studies\n",
    "weekly_returns = calculate_returns(price_data, 'weekly')\n",
    "monthly_returns = calculate_returns(price_data, 'monthly')\n",
    "\n",
    "print(f\"📊 Weekly returns shape: {weekly_returns.shape}\")\n",
    "print(f\"📊 Monthly returns shape: {monthly_returns.shape}\")\n",
    "\n",
    "# Display first few rows\n",
    "print(\"\\n📋 Sample of daily returns data:\")\n",
    "print(daily_returns.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Basic Data Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Basic statistics\n",
    "print(\"📈 BASIC STATISTICS FOR INDIAN STOCKS (2020-2022)\")\n",
    "print(\"=\"*60)\n",
    "\n",
    "# Daily returns statistics\n",
    "returns_stats = daily_returns.describe()\n",
    "print(\"\\n📊 Daily Returns Statistics:\")\n",
    "print(returns_stats.round(4))\n",
    "\n",
    "# Annualized statistics\n",
    "annual_returns = daily_returns.mean() * 252\n",
    "annual_volatility = daily_returns.std() * np.sqrt(252)\n",
    "sharpe_ratio = annual_returns / annual_volatility\n",
    "\n",
    "performance_summary = pd.DataFrame({\n",
    "    'Annual Return': annual_returns,\n",
    "    'Annual Volatility': annual_volatility,\n",
    "    'Sharpe Ratio': sharpe_ratio\n",
    "}).round(4)\n",
    "\n",
    "print(\"\\n📊 Annualized Performance Metrics:\")\n",
    "print(performance_summary.sort_values('Sharpe Ratio', ascending=False))\n",
    "\n",
    "# Best and worst performers\n",
    "print(\"\\n🏆 TOP PERFORMERS (by Sharpe Ratio):\")\n",
    "top_performers = performance_summary.sort_values('Sharpe Ratio', ascending=False).head(10)\n",
    "for i, (stock, metrics) in enumerate(top_performers.iterrows(), 1):\n",
    "    print(f\"{i:2d}. {stock:<15} - Sharpe: {metrics['Sharpe Ratio']:6.3f}, Return: {metrics['Annual Return']*100:6.2f}%\")\n",
    "\n",
    "print(\"\\n📉 WORST PERFORMERS (by Sharpe Ratio):\")\n",
    "worst_performers = performance_summary.sort_values('Sharpe Ratio', ascending=True).head(5)\n",
    "for i, (stock, metrics) in enumerate(worst_performers.iterrows(), 1):\n",
    "    print(f\"{i:2d}. {stock:<15} - Sharpe: {metrics['Sharpe Ratio']:6.3f}, Return: {metrics['Annual Return']*100:6.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔗 Correlation Analysis Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def calculate_correlation_matrix(returns_data, method='pearson'):\n",
    "    \"\"\"\n",
    "    Calculate correlation matrix with statistical significance\n",
    "    \"\"\"\n",
    "    correlation_matrix = returns_data.corr(method=method)\n",
    "    \n",
    "    # Calculate p-values for correlations\n",
    "    n = len(returns_data)\n",
    "    pvalue_matrix = pd.DataFrame(index=correlation_matrix.index, \n",
    "                                columns=correlation_matrix.columns)\n",
    "    \n",
    "    for i in correlation_matrix.index:\n",
    "        for j in correlation_matrix.columns:\n",
    "            if i != j:\n",
    "                corr_coef = correlation_matrix.loc[i, j]\n",
    "                # Calculate t-statistic and p-value\n",
    "                t_stat = corr_coef * np.sqrt((n-2)/(1-corr_coef**2))\n",
    "                p_value = 2 * (1 - stats.t.cdf(abs(t_stat), n-2))\n",
    "                pvalue_matrix.loc[i, j] = p_value\n",
    "            else:\n",
    "                pvalue_matrix.loc[i, j] = 0\n",
    "    \n",
    "    return correlation_matrix, pvalue_matrix.astype(float)\n",
    "\n",
    "def find_highly_correlated_pairs(correlation_matrix, threshold=0.7):\n",
    "    \"\"\"\n",
    "    Find pairs of stocks with high correlation\n",
    "    \"\"\"\n",
    "    high_corr_pairs = []\n",
    "    \n",
    "    for i in range(len(correlation_matrix.columns)):\n",
    "        for j in range(i+1, len(correlation_matrix.columns)):\n",
    "            stock1 = correlation_matrix.columns[i]\n",
    "            stock2 = correlation_matrix.columns[j]\n",
    "            corr_value = correlation_matrix.iloc[i, j]\n",
    "            \n",
    "            if abs(corr_value) >= threshold:\n",
    "                high_corr_pairs.append({\n",
    "                    'Stock 1': stock1,\n",
    "                    'Stock 2': stock2,\n",
    "                    'Correlation': corr_value,\n",
    "                    'Abs Correlation': abs(corr_value)\n",
    "                })\n",
    "    \n",
    "    return pd.DataFrame(high_corr_pairs).sort_values('Abs Correlation', ascending=False)\n",
    "\n",
    "def sector_correlation_analysis(returns_data, sectors_dict):\n",
    "    \"\"\"\n",
    "    Analyze correlation within and across sectors\n",
    "    \"\"\"\n",
    "    sector_correlations = {}\n",
    "    \n",
    "    for sector_name, stocks in sectors_dict.items():\n",
    "        # Filter stocks that exist in our data\n",
    "        available_stocks = [s for s in stocks if s in returns_data.columns]\n",
    "        \n",
    "        if len(available_stocks) >= 2:\n",
    "            sector_data = returns_data[available_stocks]\n",
    "            sector_corr = sector_data.corr()\n",
    "            \n",
    "            # Calculate average intra-sector correlation\n",
    "            corr_values = []\n",
    "            for i in range(len(sector_corr.columns)):\n",
    "                for j in range(i+1, len(sector_corr.columns)):\n",
    "                    corr_values.append(sector_corr.iloc[i, j])\n",
    "            \n",
    "            if corr_values:\n",
    "                sector_correlations[sector_name] = {\n",
    "                    'avg_correlation': np.mean(corr_values),\n",
    "                    'max_correlation': max(corr_values),\n",
    "                    'min_correlation': min(corr_values),\n",
    "                    'std_correlation': np.std(corr_values),\n",
    "                    'stocks_count': len(available_stocks),\n",
    "                    'stocks': available_stocks\n",
    "                }\n",
    "    \n",
    "    return pd.DataFrame(sector_correlations).T\n",
    "\n",
    "print(\"🔧 Correlation analysis functions defined successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 Overall Market Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate correlation matrix for daily returns\n",
    "print(\"🔍 OVERALL MARKET CORRELATION ANALYSIS\")\n",
    "print(\"=\"*50)\n",
    "\n",
    "correlation_matrix, pvalue_matrix = calculate_correlation_matrix(daily_returns)\n",
    "\n",
    "print(f\"📊 Correlation matrix shape: {correlation_matrix.shape}\")\n",
    "print(f\"📊 Average correlation: {correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)].mean():.4f}\")\n",
    "print(f\"📊 Median correlation: {np.median(correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)]):.4f}\")\n",
    "\n",
    "# Distribution of correlations\n",
    "corr_values = correlation_matrix.values[np.triu_indices_from(correlation_matrix.values, k=1)]\n",
    "\n",
    "print(\"\\n📈 Correlation Distribution:\")\n",
    "print(f\"Min correlation: {corr_values.min():.4f}\")\n",
    "print(f\"Max correlation: {corr_values.max():.4f}\")\n",
    "print(f\"Standard deviation: {corr_values.std():.4f}\")\n",
    "\n",
    "# Correlation ranges\n",
    "high_corr = sum(corr_values > 0.7)\n",
    "medium_corr = sum((corr_values > 0.3) & (corr_values <= 0.7))\n",
    "low_corr = sum((corr_values > 0) & (corr_values <= 0.3))\n",
    "negative_corr = sum(corr_values < 0)\n",
    "\n",
    "total_pairs = len(corr_values)\n",
    "\n",
    "print(f\"\\n📊 Correlation Ranges (out of {total_pairs} pairs):\")\n",
    "print(f\"High correlation (>0.7):     {high_corr:4d} ({high_corr/total_pairs*100:5.1f}%)\")\n",
    "print(f\"Medium correlation (0.3-0.7): {medium_corr:4d} ({medium_corr/total_pairs*100:5.1f}%)\")\n",
    "print(f\"Low correlation (0-0.3):      {low_corr:4d} ({low_corr/total_pairs*100:5.1f}%)\")\n",
    "print(f\"Negative correlation (<0):     {negative_corr:4d} ({negative_corr/total_pairs*100:5.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 High Correlation Pairs Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Find highly correlated pairs\n",
    "print(\"🔗 HIGH CORRELATION PAIRS ANALYSIS\")\n",
    "print(\"=\"*50)\n",
    "\n",
    "# Different thresholds\n",
    "thresholds = [0.8, 0.7, 0.6]\n",
    "\n",
    "for threshold in thresholds:\n",
    "    high_corr_pairs = find_highly_correlated_pairs(correlation_matrix, threshold)\n",
    "    \n",
    "    print(f\"\\n📊 Pairs with correlation >= {threshold}:\")\n",
    "    print(f\"Total pairs found: {len(high_corr_pairs)}\")\n",
    "    \n",
    "    if len(high_corr_pairs) > 0:\n",
    "        print(\"\\nTop 10 highly correlated pairs:\")\n",


    not complted